<H2>예제

&nbsp;지금까지 우리는 허밍버드 펌웨어 업데이트부터 라즈베리파이제로의 환경구축, Python을 통한 허밍버드 제어 그리고 Python에서 이미지를 처리하는 방법까지 배워왔다. 이제 간단한 이미지 처리를 통해서 드론을 직접 제어해 보는 시간을 가질 것이다. 이미지 처리를 통해서 드론을 제어하려면 어떤 상황에서 드론이 어떻게 움직일지를 먼저 정해야 한다. 다음 이미지는 드론의 하방카메라로 촬영한 비행 경로 이미지 이다.  
<br></br>

<div style="float: left; width: 33%">
<img src="./img/image_r.jpg"/>
    <center>그림 1</center>
</div>

<div style="float: left; width: 33%">
<img src="./img/image_l.jpg"/>
    <center>그림 2</center>
</div>

<div style="float: left; width: 33%">
<img src="./img/image_a.jpg"/>
    <center>그림 3</center>
</div>

&nbsp;그림 1의 경우 드론은 오른쪽으로 방향을 틀어야 하고 그림 2의 경우에는 왼쪽으로 방향을 틀어야 한다. 그리고 그림 3처럼 경로가 반듯하다면 드론은 직진할 것이다.  

+ 그림 1의 경우 => 우회전
+ 그림 2의 경우 => 좌회전
+ 그림 3의 경우 => 직진

&nbsp;우리는 이제 어떤 상황에서 드론이 어떻게 움직여야 할지 알았다. 이미지 처리를 통해서 저 세가지 조건을 구분해 드론을 제어해 보자.

## 이미지 처리

먼저 이미지 처리를 위한 기본적인 모듈을 import한다.

In [3]:
import numpy as np
import cv2
import math

그 다음 이미지 처리를 위한 실험용 주행 경로 이미지를 불러온다.

In [4]:
route_r = cv2.imread('./img/image_r.jpg')
route_l = cv2.imread('./img/image_l.jpg')
route_a = cv2.imread('./img/image_a.jpg')

우리는 불러온 세개의 이미지를 처리하여 각각 right, left, go_ahead를 출력하게 할 것이다. 따라서 for를 사용하여 세 이미지가 한 번씩 같은 코드를 거쳐 결과를 출력하게 만든다.

In [5]:
for img in [route_r, route_l, route_a]:
    pass

이제 우리는 pass 부분을 채워 넣어야 한다.

In [ ]:
import numpy as np
import cv2
import math

route_r = cv2.imread('./img/image_r.jpg')
route_l = cv2.imread('./img/image_l.jpg')
route_a = cv2.imread('./img/image_a.jpg')

for img in [route_r, route_l, route_a]:
    cv2.imshow('test', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    yuvimage = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    
    cv2.imshow('test', yuvimage)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    incrop = 150
    yuvimage = yuvimage[incrop:-150, 150:-150, :]
    
    
    cv2.imshow('test', yuvimage)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    center_idx = [80, 210]  # 세로, 가로

    u = yuvimage[:, :, 1]
    v = yuvimage[:, :, 2]

    biimage_yuv = np.zeros(np.shape(u))
    
    cv2.imshow('test', biimage_yuv)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    biimage_yuv[u > 135] = 200
    
    cv2.imshow('test', biimage_yuv)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # 파라미터 초기화
    yaw_cmd = 0
    pitch_cmd = 0
    roll_cmd = 0
    vel = 0

    road_ix = np.where(200 == biimage_yuv)
    road_x = road_ix[0]
    road_y = road_ix[1]

    yy = int(np.mean(road_x))
    xx = int(np.mean(road_y))

    top_ix = np.where(road_x[0] == road_x)
    top_i = top_ix[0][-1]
    top_max = road_y[top_i]
    bottom_max = road_y[-1]

    # ROLL 제어 Part (좌우)
    if abs(center_idx[1] - xx) > 60:  # 좌우 threshold
        if xx > 220:
            roll_cmd = -0.1  ### 오른쪽 이동 정도
        else:
            roll_cmd = 0.1  ### 왼쪽 이동 정도
        vel = 0.2
        cucmd = 'roll control (+좌,-우) : ' + str(roll_cmd)

    # YAW 제어 Part (회전)
    elif abs(top_max - bottom_max) > 50:  # 각도 threshold
        roadangle = top_max - bottom_max
        yaw_cmd = int(roadangle / 3) * (-1)
        if yaw_cmd > 35:
            yaw_cmd = 35  ### 오른쪽 최대 회전 각도
        if yaw_cmd < -35:
            yaw_cmd = -35  ### 왼쪽 최대 회전 각도
        cucmd = 'yaw control (+오,-왼) : ' + str(yaw_cmd)

    # PITCH 제어 Part (앞뒤)
    else:
        pitch_cmd = 0.25  ### 앞으로 이동할 거리
        vel = 0.3  ### 속도
        cucmd = 'pitch control (앞) : ' + str(pitch_cmd)

    print(cucmd)
#     return (0, yaw_cmd, pitch_cmd, roll_cmd, vel)